In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns.
application_df.drop(['EIN','NAME', 'ASK_AMT', 'SPECIAL_CONSIDERATIONS'],axis=1,inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
apptypecount = application_df['APPLICATION_TYPE'].value_counts()
apptypecount

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(apptypecount[apptypecount<520].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classcount = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION']
                                                                  .value_counts() >= 1]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classcount[classcount < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
df_edit = pd.get_dummies(application_df,columns=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT'])
df_edit.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [7]:
# Split our preprocessed data into our features and target arrays
y = df_edit['IS_SUCCESSFUL'].values
X = df_edit.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden1 = 80
hidden2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=20, input_dim=input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                840       
                                                                 
 dense_1 (Dense)             (None, 15)                315       
                                                                 
 dense_2 (Dense)             (None, 10)                160       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,326
Trainable params: 1,326
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model
model = nn.fit(X_train, y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 1s 539us/step - loss: 0.5845 - accuracy: 0.7146
Epoch 2/25
804/804 [==============================] - 0s 541us/step - loss: 0.5629 - accuracy: 0.7276
Epoch 3/25
804/804 [==============================] - 0s 533us/step - loss: 0.5582 - accuracy: 0.7278
Epoch 4/25
804/804 [==============================] - 0s 539us/step - loss: 0.5565 - accuracy: 0.7289
Epoch 5/25
804/804 [==============================] - 0s 538us/step - loss: 0.5552 - accuracy: 0.7294
Epoch 6/25
804/804 [==============================] - 0s 542us/step - loss: 0.5534 - accuracy: 0.7313
Epoch 7/25
804/804 [==============================] - 0s 542us/step - loss: 0.5524 - accuracy: 0.7308
Epoch 8/25
804/804 [==============================] - 0s 533us/step - loss: 0.5519 - accuracy: 0.7308
Epoch 9/25
804/804 [==============================] - 0s 534us/step - loss: 0.5518 - accuracy: 0.7318
Epoch 10/25
804/804 [==============================] - 0s 534us/step - loss: 0.550

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6306 - accuracy: 0.6896 - 173ms/epoch - 646us/step
Loss: 0.6305749416351318, Accuracy: 0.6895626783370972


In [13]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")